<a href="https://colab.research.google.com/github/SoudeepGhoshal/ColabFiltering/blob/main/PIP_CollaborativeFiltering_MovieLens100k_KNNImputed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
cols_u = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-100k/u.user', sep='|', names=cols_u, encoding='latin-1', parse_dates=True)

ratings_u = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-100k/u.data', sep='\t', names=ratings_u, encoding='latin-1')

movies_u = ['movie_id' , 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-100k/u.item', sep='|', names=movies_u, usecols=range(5), encoding='latin-1')

movie_ratings=pd.merge(movies,ratings)
movie_df=pd.merge(movie_ratings,users)

movie_df.head(500)

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076
...,...,...,...,...,...,...,...,...,...,...,...,...
495,214,Pink Floyd - The Wall (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Pink%20Floyd%...,148,5,877019882,33,M,engineer,97006
496,222,Star Trek: First Contact (1996),22-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,148,4,877398901,33,M,engineer,97006
497,227,Star Trek VI: The Undiscovered Country (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Star%20Trek%2...,148,4,877399083,33,M,engineer,97006
498,228,Star Trek: The Wrath of Khan (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,148,4,877016514,33,M,engineer,97006


In [52]:
#only for the normalization
min_rating = ratings['rating'].min()
max_rating = ratings['rating'].max()

ratings['rating'] = (ratings['rating'] - min_rating) / (max_rating - min_rating)

ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,0.50,881250949
1,186,302,0.50,891717742
2,22,377,0.00,878887116
3,244,51,0.25,880606923
4,166,346,0.00,886397596


In [53]:
movie_df.drop(movie_df.columns[[2,5]], axis=1, inplace=True)
ratings.drop("timestamp", inplace=True, axis=1)
movies.drop(movies.columns[[2]], inplace=True, axis=1)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            100000 non-null  int64  
 1   title               100000 non-null  object 
 2   video_release_date  0 non-null       float64
 3   imdb_url            99987 non-null   object 
 4   rating              100000 non-null  int64  
 5   timestamp           100000 non-null  int64  
 6   age                 100000 non-null  int64  
 7   sex                 100000 non-null  object 
 8   occupation          100000 non-null  object 
 9   zip_code            100000 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 10.4+ MB


Run if not using Imputation on ratings

In [ ]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings.sample(n=num_user_input, random_state=42)
subset_ratings = ratings.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
895536,5412,2683,0.25
899739,5440,904,1.00
55687,368,3717,0.75
63727,425,1721,0.75
822011,4942,3697,0.00
...,...,...,...
85492,558,923,1.00
586763,3582,329,0.75
589709,3597,3175,0.50
548798,3389,2987,0.75


Run for Imputation

In [54]:
userItemMatrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
userItemMatrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.00,0.5,0.75,0.50,0.5,1.0,0.75,0.0,1.0,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.75,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,0.25,NaN,NaN,0.75,1.0,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,1.00,NaN,NaN,NaN,NaN,NaN,0.75,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
null_values = userItemMatrix.isnull().sum().sum()
print(f"Total number of null values: {null_values}")

Total number of null values: 1486126


Simple Imputer technique applied (Mean/Median)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Iterate over each row
for index, row in userItemMatrix_imputed.iterrows():
    # Calculate the number of null values in each row
    row_null_values = row.isnull().sum()

    # If the number of null values in the row is less than the threshold, skip imputation for this row
    if row_null_values == 0 or row_null_values > null_threshold:
        continue

    # Create a mask for 60% of the null values in this row
    row_mask = np.random.choice([True, False], len(row), p=[0.6, 0.4])

    # Apply imputation for the selected 60% of null values in this row
    row_values = row.values
    row_values[row_mask] = imputer.fit_transform(row_values[row_mask].reshape(-1, 1)).ravel()

    # Update the values in the DataFrame for this row
    userItemMatrix_imputed.loc[index] = row_values

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 60.00%
Remaining null values: 8554519


KNN Imputation technique applied

In [56]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Perform kNN imputation on the DataFrame
userItemMatrix_imputed = pd.DataFrame(knn_imputer.fit_transform(userItemMatrix_imputed))

# Create a random mask to identify positions for replacement
mask = np.random.choice([True, False], size=userItemMatrix_imputed.shape, p=[0.4, 0.6])

# Update the corresponding values in the DataFrame with np.nan
userItemMatrix_imputed[mask] = np.nan

# Display the DataFrame after imputation
userItemMatrix_imputed

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,1.0,NaN,0.75,0.50,0.50,1.00,0.75,0.00,1.00,NaN,...,0.5,0.75,0.5,0.25,NaN,NaN,NaN,NaN,0.5,0.5
1,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.95,0.85,0.25,...,0.5,0.75,NaN,0.25,0.5,0.0,0.5,0.25,0.5,0.5
2,NaN,0.6,NaN,NaN,0.60,NaN,NaN,0.80,0.80,0.80,...,0.5,0.75,NaN,0.25,NaN,0.0,NaN,0.25,NaN,NaN
3,NaN,0.5,NaN,0.45,0.55,NaN,0.85,0.80,NaN,0.90,...,0.5,0.75,NaN,0.25,0.5,0.0,0.5,NaN,NaN,NaN
4,NaN,0.5,0.55,NaN,0.60,NaN,0.80,NaN,0.80,0.70,...,NaN,NaN,0.5,0.25,NaN,NaN,NaN,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.8,0.3,NaN,NaN,0.75,NaN,0.90,NaN,1.00,0.70,...,0.5,0.75,0.5,0.25,NaN,0.0,0.5,0.25,NaN,NaN
939,0.6,NaN,0.65,0.25,NaN,0.75,0.75,1.00,NaN,NaN,...,0.5,0.75,0.5,NaN,NaN,NaN,0.5,0.25,NaN,NaN
940,1.0,NaN,0.50,0.65,NaN,0.60,0.75,NaN,0.85,0.75,...,0.5,0.75,0.5,NaN,0.5,0.0,0.5,0.25,0.5,0.5
941,NaN,NaN,0.70,0.70,0.75,NaN,0.50,NaN,0.80,0.60,...,0.5,0.75,0.5,0.25,NaN,0.0,0.5,0.25,0.5,0.5


In [57]:
# Calculate the total number of values in the DataFrame
total_values = userItemMatrix_imputed.size

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 57.26%
Remaining null values: 635119


In [58]:
#cpmare table sizes for any anomaly
print(userItemMatrix_imputed.shape[1])
print(userItemMatrix.shape[1])

1682
1682


In [59]:
print(len(ratings))

100000


In [60]:
ratings_series = userItemMatrix_imputed.stack()
ratings_imputed = ratings_series.reset_index()
ratings_imputed.columns = ['user_id', 'movie_id', 'rating']

In [61]:
print(len(ratings_imputed))

951007


In [62]:
#Testing for new values
userItemMatrix_imputed

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,1.0,NaN,0.75,0.50,0.50,1.00,0.75,0.00,1.00,NaN,...,0.5,0.75,0.5,0.25,NaN,NaN,NaN,NaN,0.5,0.5
1,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.95,0.85,0.25,...,0.5,0.75,NaN,0.25,0.5,0.0,0.5,0.25,0.5,0.5
2,NaN,0.6,NaN,NaN,0.60,NaN,NaN,0.80,0.80,0.80,...,0.5,0.75,NaN,0.25,NaN,0.0,NaN,0.25,NaN,NaN
3,NaN,0.5,NaN,0.45,0.55,NaN,0.85,0.80,NaN,0.90,...,0.5,0.75,NaN,0.25,0.5,0.0,0.5,NaN,NaN,NaN
4,NaN,0.5,0.55,NaN,0.60,NaN,0.80,NaN,0.80,0.70,...,NaN,NaN,0.5,0.25,NaN,NaN,NaN,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.8,0.3,NaN,NaN,0.75,NaN,0.90,NaN,1.00,0.70,...,0.5,0.75,0.5,0.25,NaN,0.0,0.5,0.25,NaN,NaN
939,0.6,NaN,0.65,0.25,NaN,0.75,0.75,1.00,NaN,NaN,...,0.5,0.75,0.5,NaN,NaN,NaN,0.5,0.25,NaN,NaN
940,1.0,NaN,0.50,0.65,NaN,0.60,0.75,NaN,0.85,0.75,...,0.5,0.75,0.5,NaN,0.5,0.0,0.5,0.25,0.5,0.5
941,NaN,NaN,0.70,0.70,0.75,NaN,0.50,NaN,0.80,0.60,...,0.5,0.75,0.5,0.25,NaN,0.0,0.5,0.25,0.5,0.5


In [63]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings_imputed)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings_imputed.sample(n=num_user_input, replace=True, random_state=42)
subset_ratings = ratings_imputed.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
121958,120,1006,0.75
671155,665,584,0.35
131932,130,807,0.50
365838,362,1138,0.15
259178,256,773,0.70
...,...,...,...
925223,917,828,0.35
431257,427,706,0.90
643397,637,1060,0.70
816385,809,808,0.55


Imputation portion ends here

In [ ]:
# movies['title'] = movies['title'].str.replace(r'\(\d\d\d\d\)', '')
# movies['title'] = movies['title'].apply(lambda x: x.strip())
# movies.head()

In [ ]:
# userInput = [
#             {'title':'Breakfast Club, The', 'rating':5},
#             {'title':'Toy Story', 'rating':3.5},
#             {'title':'Jumanji', 'rating':2},
#             {'title':"Pulp Fiction", 'rating':5},
#             {'title':'Akira', 'rating':4.5}
#          ]
# inputMovies = pd.DataFrame(userInput)
# inputMovies

In [ ]:
# # inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
# # inputMovies = pd.merge(inputId, inputMovies)
# inputMovies = inputMovies.drop('release_date', 1)
# inputMovies

In [64]:
userSubset = subset_ratings[subset_ratings['movie_id'].isin(user_input_ratings['movie_id'].tolist())]
userSubset.head()

,user_id,movie_id,rating
0,0,0,1.0
2,0,3,0.5
4,0,5,1.0
6,0,7,0.0
7,0,8,1.0


In [65]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup.get_group(222)
mu_k=userSubsetGroup['rating'].mean()

In [66]:
userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

<ipython-input-66-5489e10f2460>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)


[(382,
          user_id  movie_id  rating
  385423      382         1    0.55
  385424      382         4    0.80
  385425      382         6    0.50
  385426      382         7    0.65
  385427      382         8    1.00
  ...         ...       ...     ...
  386438      382      1677    0.00
  386439      382      1678    0.50
  386440      382      1679    0.25
  386441      382      1680    0.50
  386442      382      1681    0.50
  
  [810 rows x 3 columns]),
 (193,
          user_id  movie_id  rating
  194991      193         3    0.75
  194992      193         4    0.50
  194993      193         7    0.50
  194996      193        13    0.80
  194997      193        14    0.75
  ...         ...       ...     ...
  196023      193      1673    0.75
  196024      193      1674    0.50
  196026      193      1676    0.50
  196027      193      1679    0.25
  196028      193      1681    0.50
  
  [809 rows x 3 columns]),
 (908,
          user_id  movie_id  rating
  915612      908  

In [67]:
userSubsetGroup=userSubsetGroup[0:500]

In [68]:
ratings_median=ratings['rating'].median()
print(ratings_median)

0.75


In [69]:
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    agree = agreement(rating1, rating2, rating_median)
    dist = np.where(agree, np.absolute(rating1 - rating2), 2 * np.absolute(rating1 - rating2))
    prox = ((2 * (rating_max - rating_min) + 1) - dist) ** 2
    return prox

def impact(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    impact_score = (np.absolute(rating1 - rating_median) + 1) * (np.absolute(rating2 - rating_median) + 1)
    agree = agreement(rating1, rating2, rating_median)
    impact_result = np.where(agree, impact_score, 1 / impact_score)
    return impact_result

def popularity(rating1, rating2, mu_k):
    pop = np.where(
        (rating1 > mu_k) & (rating2 > mu_k) | (rating1 < mu_k) & (rating2 < mu_k),
        1 + ((rating1 + rating2) / 2 - mu_k) ** 2,
        1
    )
    return pop

In [70]:
import math
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    D = np.abs(rating1 - rating2)
    med_plus = (rating_max + rating_median) / 2
    med_minus = (rating_min + rating_median) / 2
    Rmax_minus_Rmin_squared = (rating_max - rating_min) ** 2

    agree = agreement(rating1, rating2, rating_median)
    prox = (D - med_plus + med_minus) ** 2 / Rmax_minus_Rmin_squared
    delta = 0.75 if np.abs(rating1 - rating2) > rating_median else (0.5 if np.abs(rating1 - rating2) == rating_median else 0.25)
    return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))

def impact(rating1, rating2, rating_median):
    agree = agreement(rating1, rating2, rating_median)
    impact1= math.exp(-1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    impact2= math.exp(1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    return impact1 if agree else impact2

def proposed_popularity(rating1, rating2,rIi):
    if (rating1 > rIi and rating2 > rIi) or (rating1 < rIi and rating2 < rIi):
        return math.log10(2 + (((rating1 + rating2) / 2) - rIi) ** 2)
    else:
        return 0.3010


In [72]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:

    group = group.sort_values(by='movie_id')
    inputMovies = user_input_ratings.sort_values(by='movie_id')
    temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    pip_similarity_value = pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, ratings_median, mu_k)
    PIPdict[name] = pip_similarity_value.mean()

print(sorted(PIPdict.keys()))

<ipython-input-70-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in scalar divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[1, 3, 8, 10, 12, 13, 15, 16, 17, 20, 21, 23, 25, 30, 32, 34, 35, 36, 37, 39, 40, 44, 45, 46, 47, 52, 54, 57, 60, 61, 63, 64, 66, 68, 70, 71, 72, 77, 78, 79, 80, 82, 84, 85, 86, 87, 88, 89, 91, 93, 98, 100, 101, 102, 103, 104, 106, 108, 110, 118, 119, 120, 122, 128, 130, 132, 133, 134, 136, 137, 138, 140, 145, 147, 148, 149, 150, 151, 152, 153, 155, 157, 158, 159, 160, 161, 163, 164, 165, 166, 168, 171, 173, 176, 177, 181, 182, 183, 186, 187, 190, 192, 193, 194, 195, 196, 199, 200, 201, 204, 205, 206, 207, 208, 211, 212, 213, 214, 216, 221, 225, 226, 227, 229, 230, 231, 233, 236, 242, 244, 245, 246, 247, 248, 250, 252, 254, 259, 261, 263, 265, 267, 269, 271, 274, 281, 283, 285, 288, 290, 294, 295, 296, 300, 301, 303, 305, 307, 308, 313, 314, 318, 319, 321, 323, 324, 327, 328, 329, 330, 333, 334, 338, 341, 344, 350, 351, 353, 354, 357, 358, 359, 360, 364, 366, 367, 369, 371, 374, 375, 376, 377, 378, 379, 381, 382, 383, 384, 385, 386, 388, 389, 390, 393, 397, 398, 399, 401, 402, 404, 406

In [73]:
print(PIPdict[1])

inf


In [74]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:
  group = group.sort_values(by='movie_id')
  inputMovies = user_input_ratings.sort_values(by='movie_id')
  temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
  tempRatingList = temp_df['rating'].tolist()
  tempGroupList = group['rating'].tolist()

  similarity_values = []

  for temp_rating, group_rating in zip(tempRatingList, tempGroupList):
      pip_similarity_value = pip_similarity([temp_rating], [group_rating], rating_max, rating_min, ratings_median, mu_k)
      similarity_values.append(pip_similarity_value)

print(similarity_values)

<ipython-input-70-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in scalar divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[0.08902163713969871, 0.08513914614978273, 0.08934831751199872, inf, 0.08902163713969871, 0.10952779758546313, inf, 0.07730991090286225, 0.24820650846230127, 0.08358158041774556, 0.08934831751199872, 0.04510149128355261, 0.11504879454464127, inf, 0.08934831751199872, inf, 0.11504879454464127, 0.0442895147477107, inf, 0.059928246699386394, 0.4092676153766133, 0.2085216917618414, 0.049954576732322724, 0.23119878156522705, 0.1421870180334092, 0.08902163713969871, 0.08436678784688777, 0.06708216887960425, 0.08436678784688777, 0.03895468082109352, 0.07193685113161484, 0.09036792684776553, inf, 0.02855687546858048, 0.09036792684776553, 0.08797933608246876, inf, 0.5183746891629745, 0.08712002550995482, 0.08902163713969871, 0.24820650846230127, inf, 0.5183746891629745, 0.035979978784176976, 0.04722119873160414, 0.19714519245691786, 0.44779493223641975, 0.5183746891629745, 0.08902163713969871, 0.08436678784688777, 0.08902163713969871, 0.24820650846230127, 0.09293582903639577, 0.1590582194513269

In [75]:
#PIP_DF = pd.DataFrame.from_dict(PIPdict, orient='index')
PIP_DF = pd.DataFrame(similarity_values, columns=['Similarity'])
PIP_DF.columns = ['similarityIndex']
PIP_DF['userId'] = PIP_DF.index
PIP_DF.index = range(len(PIP_DF))

#removing the inf values

PIP_DF['similarityIndex'] = PIP_DF['similarityIndex'].replace([np.inf, -np.inf], 0)
PIP_DF.head()

,similarityIndex,userId
0,0.089022,0
1,0.085139,1
2,0.089348,2
3,0.000000,3
4,0.089022,4


In [76]:
topUsers=PIP_DF.sort_values(by='similarityIndex', ascending=False)
topUsers.head(100)

,similarityIndex,userId
715,0.908797,715
648,0.775607,648
37,0.518375,37
57,0.518375,57
157,0.518375,157
...,...,...
635,0.302043,635
725,0.261999,725
392,0.261999,392
644,0.261999,644


In [77]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='user_id', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
#topUsersRating.head(18000)
topUsersRating.head(1352)
topUsersRating.isnull().sum()

similarityIndex    0
userId             0
user_id            0
movie_id           0
rating             0
weightedRating     0
dtype: int64

In [78]:
tempTopUsersRating = topUsersRating.groupby('movie_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
#handled the zero division error bug
recommendation_df.loc[tempTopUsersRating['sum_similarityIndex'] == 0, 'weighted average recommendation score'] = 0
recommendation_df.fillna(0, inplace=True)
##
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.tail()

,weighted average recommendation score,movieID
movie_id,,
1654,0.00,1654
1655,0.25,1655
1656,0.25,1656
1657,0.00,1657
1658,0.00,1658


In [79]:
tempTopUsersRating.head(973)

,sum_similarityIndex,sum_weightedRating
movie_id,,
1,53.607802,40.216912
2,14.819841,8.445942
3,13.226771,6.921246
4,26.393888,16.470835
5,10.597519,6.427504
...,...,...
969,9.834600,6.494757
970,1.138929,0.659364
971,3.593844,2.332008


In [80]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(100)

,weighted average recommendation score,movieID
movie_id,,
1189,1.000000,1189
1122,1.000000,1122
1467,1.000000,1467
1463,1.000000,1463
1599,1.000000,1599
...,...,...
645,0.787185,645
525,0.786312,525
168,0.785194,168


In [81]:
recommendation_df = recommendation_df[(recommendation_df['weighted average recommendation score'] <= 5) &
 (recommendation_df['weighted average recommendation score'] >= 0) &
 (recommendation_df['weighted average recommendation score'].notnull())]
print(recommendation_df)

          weighted average recommendation score  movieID
movie_id                                                
1189                                        1.0     1189
1122                                        1.0     1122
1467                                        1.0     1467
1463                                        1.0     1463
1599                                        1.0     1599
...                                         ...      ...
1433                                        0.0     1433
1486                                        0.0     1486
1493                                        0.0     1493
1494                                        0.0     1494
1658                                        0.0     1658

[1658 rows x 2 columns]


In [82]:
print(recommendation_df.isnull().sum())

weighted average recommendation score    0
movieID                                  0
dtype: int64


In [83]:
recommendation_df.head(10)

,weighted average recommendation score,movieID
movie_id,,
1189,1.0,1189
1122,1.0,1122
1467,1.0,1467
1463,1.0,1463
1599,1.0,1599
1293,1.0,1293
1191,1.0,1191
1201,1.0,1201
1500,1.0,1500


In [84]:
movies.loc[movies['movie_id'].isin(recommendation_df.head(10)['movieID'].tolist())]

,movie_id,title,video_release_date,imdb_url
1121,1122,They Made Me a Criminal (1939),NaN,http://us.imdb.com/M/title-exact?They%20Made%2...
1188,1189,Prefontaine (1997),NaN,http://us.imdb.com/M/title-exact?Prefontaine%2...
1190,1191,"Letter From Death Row, A (1998)",NaN,http://us.imdb.com/M/title-exact?Letter+From+D...
1200,1201,Marlene Dietrich: Shadow and Light (1996),NaN,http://us.imdb.com/M/title-exact?Marlene%20Die...
1292,1293,Star Kid (1997),NaN,http://us.imdb.com/M/title-exact?imdb-title-12...
1462,1463,"Boys, Les (1997)",NaN,http://us.imdb.com/M/title-exact?imdb-title-11...
1466,1467,"Saint of Fort Washington, The (1993)",NaN,http://us.imdb.com/M/title-exact?Saint%20of%20...
1499,1500,Santa with Muscles (1996),NaN,http://us.imdb.com/M/title-exact?Santa%20with%...
1535,1536,Aiqing wansui (1994),NaN,http://us.imdb.com/M/title-exact?Aiqing%20Wans...
1598,1599,Someone Else's America (1995),NaN,http://us.imdb.com/M/title-exact?Someone%20Els...


In [85]:
movies.head()

,movie_id,title,video_release_date,imdb_url
0,1,Toy Story (1995),NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [86]:
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings,on='movie_id')
merged_ratings['squared_diff'] = (merged_ratings['weighted average recommendation score'] - merged_ratings['rating']) ** 2
mean_squared_diff = merged_ratings['squared_diff'].mean()
mean_squared_diff
rmse_score = sqrt(mean_squared_diff)
print("RMSE Score:", rmse_score)

RMSE Score: 0.25276464770666623


In [87]:
import pandas as pd
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings, on='movie_id')

merged_ratings['abs_diff'] = abs(merged_ratings['weighted average recommendation score'] - merged_ratings['rating'])

mae_score = merged_ratings['abs_diff'].mean()

print("MAE Score:", mae_score)

MAE Score: 0.20128109352566742
